In [2]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

im_drone = Image.open('drone4-1.png')
im_drone = im_drone.resize((20,20))

imb = Image.open('bullet.png')
imb = imb.resize((5,5))




import cv2
import numpy as np

cap = cv2.VideoCapture('video_bullet.avi')

uframes = []
ret, frame = cap.read()
uframes.append(frame)
flag = 0
z = 0
while(z<10):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    
    for a in uframes:
        if np.array(frame).any() != None:
            difference = cv2.subtract(a, frame)    
            result = not np.any(difference)
            print("inside for")
            if result is True:
                flag = 1
                print("setting flag")
                break
    if(not flag):
        print("appending")
        uframes.append(frame)
    z+=1


# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

for i in range(samples):
    switch = random.randint(0,1)
    if switch==0:
        x = random.randint(0,1)
        if x==0:
            data1.append(np.array(np.transpose(np.asarray(uframes[1]),(2,0,1)),dtype=np.float32))
            data2.append(np.array(np.transpose(np.asarray(uframes[-1]),(2,0,1)),dtype=np.float32))
        elif x==1:
            data1.append(np.array(np.transpose(np.asarray(uframes[2]),(2,0,1)),dtype=np.float32))
            data2.append(np.array(np.transpose(np.asarray(uframes[-1]),(2,0,1)),dtype=np.float32))
        #mg.show()
        
    elif switch==1:
        data1.append(np.array(np.transpose(np.asarray(uframes[0]),(2,0,1)),dtype=np.float32))
        data2.append(np.array(np.transpose(np.asarray(uframes[0]),(2,0,1)),dtype=np.float32))

    

# for i in range(samples):
    
#     switch = random.randint(0,1)
#     if switch==0:
#         img = Image.new('RGB', (100, 100), color = '#9b7653')
#         img.paste(im1,(5,75),im1)
#         img.paste(im1,(10,55),im1)
#         img.paste(im1,(25,50),im1)
#         img.paste(im2,(75,5),im2)
#         img.paste(im2,(60,30),im2)
#         img.paste(im2,(90,20),im2)
#         img.paste(im3,(75,60),im3)
#         img.paste(im3,(60,80),im3)
        
#         img_b = Image.new('RGB', (100, 100), color = '#9b7653')
#         img_b.paste(im1,(5,75),im1)
#         img_b.paste(im1,(10,55),im1)
#         img_b.paste(im1,(25,50),im1)
#         img_b.paste(im2,(75,5),im2)
#         img_b.paste(im2,(60,30),im2)
#         img_b.paste(im2,(90,20),im2)
#         img_b.paste(im3,(75,60),im3)
#         img_b.paste(im3,(60,80),im3)
        
#         x = random.randint(0,1)
#         if x==0:
#             img.paste(im_drone,(5,5),im_drone)
#             data1.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
#             img_b.paste(im_drone,(72,65),im_drone)
#             img_b.paste(imb,(80,80),imb)
#             data2.append(np.array(np.transpose(np.asarray(img_b),(2,0,1)),dtype=np.float32))
#         elif x==1:
#             img.paste(im_drone,(40,40),im_drone)
#             draw = ImageDraw.Draw(img)
#             draw.ellipse((50,50,55,55), fill=(0, 0, 255))
#             data1.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
#             img_b.paste(im_drone,(72,65),im_drone)
#             img_b.paste(imb,(80,80),imb)
#             data2.append(np.array(np.transpose(np.asarray(img_b),(2,0,1)),dtype=np.float32))
#         #mg.show()
        
#     elif switch==1:
#         img_blank = Image.new('RGB', (100, 100), color = '#9b7653')
#         img_blank.paste(im1,(5,75),im1)
#         img_blank.paste(im1,(10,55),im1)
#         img_blank.paste(im1,(25,50),im1)
#         img_blank.paste(im2,(75,5),im2)
#         img_blank.paste(im2,(60,30),im2)
#         img_blank.paste(im2,(90,20),im2)
#         img_blank.paste(im3,(75,60),im3)
#         img_blank.paste(im3,(60,80),im3)
#         #mg_blank.show()
#         data1.append(np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32))
#         data2.append(np.array(np.transpose(np.asarray(img_blank),(2,0,1)),dtype=np.float32))


#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 3)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


inside for
appending
inside for
inside for
appending
inside for
inside for
inside for
appending
inside for
setting flag
inside for
inside for
setting flag
inside for
inside for
inside for
setting flag
inside for
inside for
inside for
inside for
setting flag
inside for
setting flag
inside for
inside for
setting flag
inside for
inside for
inside for
setting flag
(20000, 3, 100, 100)
epoch [1/200], loss:-1.0000
x:  tensor([[  7.9709, -11.5213, -14.3839]], grad_fn=<AddmmBackward>)
y: tensor([[  7.9709, -11.5213, -14.3839]], grad_fn=<AddmmBackward>)
b: tensor([[  7.5796, -10.0668, -11.8312]], grad_fn=<AddmmBackward>)
epoch [2/200], loss:-0.9987
x:  tensor([[12.5365,  6.1163, -9.7720]], grad_fn=<AddmmBackward>)
y: tensor([[12.5365,  6.1163, -9.7720]], grad_fn=<AddmmBackward>)
b: tensor([[11.0550,  6.3308, -7.8214]], grad_fn=<AddmmBackward>)
epoch [3/200], loss:-0.9999
x:  tensor([[-9.2529, -8.9929, -6.9955]], grad_fn=<AddmmBackward>)
y: tensor([[-9.2208, -8.9339, -7.0689]], grad_fn=<AddmmBac

epoch [35/200], loss:-0.9997
x:  tensor([[-19.1176,  36.4511,   2.1130]], grad_fn=<AddmmBackward>)
y: tensor([[-18.9880,  36.7918,   2.3577]], grad_fn=<AddmmBackward>)
b: tensor([[-13.7306,  30.6829,   1.1175]], grad_fn=<AddmmBackward>)
epoch [36/200], loss:-0.9988
x:  tensor([[-7.6367, 31.3419, -7.1304]], grad_fn=<AddmmBackward>)
y: tensor([[-7.6367, 31.3419, -7.1304]], grad_fn=<AddmmBackward>)
b: tensor([[-6.1232, 26.4762, -6.9199]], grad_fn=<AddmmBackward>)
epoch [37/200], loss:-0.9999
x:  tensor([[-17.4570,  33.0704,  -9.5704]], grad_fn=<AddmmBackward>)
y: tensor([[-17.2734,  33.3248,  -9.3957]], grad_fn=<AddmmBackward>)
b: tensor([[-12.3083,  28.1133,  -8.6621]], grad_fn=<AddmmBackward>)
epoch [38/200], loss:-1.0000
x:  tensor([[-16.6447,  10.2036, -14.3336]], grad_fn=<AddmmBackward>)
y: tensor([[-16.4568,  10.4260, -14.1749]], grad_fn=<AddmmBackward>)
b: tensor([[-11.6079,   8.8012, -13.0890]], grad_fn=<AddmmBackward>)
epoch [39/200], loss:-0.9993
x:  tensor([[-9.8824, 14.1878, -

epoch [71/200], loss:-0.9995
x:  tensor([[-8.5531,  4.2918, -2.7870]], grad_fn=<AddmmBackward>)
y: tensor([[-8.5531,  4.2918, -2.7870]], grad_fn=<AddmmBackward>)
b: tensor([[-6.6616,  2.9000, -2.9852]], grad_fn=<AddmmBackward>)
epoch [72/200], loss:-0.9978
x:  tensor([[-15.2393,   4.7773,  -3.3103]], grad_fn=<AddmmBackward>)
y: tensor([[-15.0250,   4.7735,  -3.3119]], grad_fn=<AddmmBackward>)
b: tensor([[-10.2391,   3.3400,  -3.4090]], grad_fn=<AddmmBackward>)
epoch [73/200], loss:-0.9991
x:  tensor([[-5.9566,  4.5902, -3.2254]], grad_fn=<AddmmBackward>)
y: tensor([[-5.7492,  4.5833, -3.2096]], grad_fn=<AddmmBackward>)
b: tensor([[-2.7688,  3.1758, -3.3420]], grad_fn=<AddmmBackward>)
epoch [74/200], loss:-0.9988
x:  tensor([[-20.8017,   0.9749,   2.2336]], grad_fn=<AddmmBackward>)
y: tensor([[-20.6088,   0.9698,   2.2033]], grad_fn=<AddmmBackward>)
b: tensor([[-15.1530,   0.0513,   1.3518]], grad_fn=<AddmmBackward>)
epoch [75/200], loss:-0.9985
x:  tensor([[-18.0470,   3.9084,  -2.8206

epoch [107/200], loss:-0.9995
x:  tensor([[-3.8009,  4.6253, -2.8575]], grad_fn=<AddmmBackward>)
y: tensor([[-3.8009,  4.6253, -2.8575]], grad_fn=<AddmmBackward>)
b: tensor([[-2.8353,  2.7660, -2.9348]], grad_fn=<AddmmBackward>)
epoch [108/200], loss:-1.0000
x:  tensor([[-5.7223,  5.1527, -4.4811]], grad_fn=<AddmmBackward>)
y: tensor([[-5.5668,  5.1414, -4.3726]], grad_fn=<AddmmBackward>)
b: tensor([[-2.7413,  3.1977, -4.1744]], grad_fn=<AddmmBackward>)
epoch [109/200], loss:-0.9999
x:  tensor([[-10.2148,  16.2878,  -2.4110]], grad_fn=<AddmmBackward>)
y: tensor([[-10.0620,  16.3589,  -2.4390]], grad_fn=<AddmmBackward>)
b: tensor([[-6.2483, 12.5726, -2.5281]], grad_fn=<AddmmBackward>)
epoch [110/200], loss:-1.0000
x:  tensor([[-6.5340,  5.3924, -2.7911]], grad_fn=<AddmmBackward>)
y: tensor([[-6.3864,  5.3972, -2.7901]], grad_fn=<AddmmBackward>)
b: tensor([[-3.2876,  3.3569, -2.8652]], grad_fn=<AddmmBackward>)
epoch [111/200], loss:-0.9988
x:  tensor([[-9.9963,  5.0216,  6.9617]], grad_f

epoch [143/200], loss:-0.9991
x:  tensor([[-10.4505,   4.0654,  -2.3417]], grad_fn=<AddmmBackward>)
y: tensor([[-10.3688,   4.0650,  -2.3388]], grad_fn=<AddmmBackward>)
b: tensor([[-7.6752,  2.2652, -2.3808]], grad_fn=<AddmmBackward>)
epoch [144/200], loss:-0.9991
x:  tensor([[-14.9647,   5.4730,  -1.7366]], grad_fn=<AddmmBackward>)
y: tensor([[-14.9094,   5.4771,  -1.7403]], grad_fn=<AddmmBackward>)
b: tensor([[-11.0498,   3.4601,  -1.8423]], grad_fn=<AddmmBackward>)
epoch [145/200], loss:-0.9995
x:  tensor([[-10.5873,   6.2379,   2.2979]], grad_fn=<AddmmBackward>)
y: tensor([[-10.5118,   6.2909,   2.2509]], grad_fn=<AddmmBackward>)
b: tensor([[-7.5210,  4.0010,  1.5930]], grad_fn=<AddmmBackward>)
epoch [146/200], loss:-1.0000
x:  tensor([[-8.4696,  4.8418, -1.9904]], grad_fn=<AddmmBackward>)
y: tensor([[-8.3845,  4.8422, -1.9898]], grad_fn=<AddmmBackward>)
b: tensor([[-6.1102,  2.8631, -1.9973]], grad_fn=<AddmmBackward>)
epoch [147/200], loss:-0.9994
x:  tensor([[-10.9392,   4.5720, 

epoch [179/200], loss:-0.9904
x:  tensor([[-12.5366,   4.8075,  -1.8186]], grad_fn=<AddmmBackward>)
y: tensor([[-12.5562,   4.8074,  -1.8083]], grad_fn=<AddmmBackward>)
b: tensor([[-9.0676,  3.3159, -1.6714]], grad_fn=<AddmmBackward>)
epoch [180/200], loss:-0.9994
x:  tensor([[-12.7824,   4.3215,  -1.6041]], grad_fn=<AddmmBackward>)
y: tensor([[-12.8126,   4.3166,  -1.6120]], grad_fn=<AddmmBackward>)
b: tensor([[-9.6489,  2.9141, -1.5698]], grad_fn=<AddmmBackward>)
epoch [181/200], loss:-0.9998
x:  tensor([[-14.1811,   5.1110,  -1.8429]], grad_fn=<AddmmBackward>)
y: tensor([[-14.2113,   5.1082,  -1.8440]], grad_fn=<AddmmBackward>)
b: tensor([[-10.7474,   3.5874,  -1.7058]], grad_fn=<AddmmBackward>)
epoch [182/200], loss:-0.9998
x:  tensor([[-6.6588,  5.0947, -2.0531]], grad_fn=<AddmmBackward>)
y: tensor([[-6.6588,  5.0947, -2.0531]], grad_fn=<AddmmBackward>)
b: tensor([[-6.1469,  3.5975, -1.8638]], grad_fn=<AddmmBackward>)
epoch [183/200], loss:-0.9989
x:  tensor([[-5.6263,  5.1303, -2

In [2]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [3]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [4]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [5]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [6]:
import random
random.shuffle(class_data)




In [15]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets


def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out


# import some data to play with

# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
for i in range(2):
    img = uframes[3]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[-6.018177   2.2968793 -1.302309 ]]
[1]
[[-6.018177   2.2968793 -1.302309 ]]
[1]


In [8]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]
[[-2.4500804   1.5547009  -0.36525798]]
[0]


In [1]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'Image' is not defined

In [13]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
[[-3.5312743   1.5332453  -0.45803857]]
[0]
